# To do 
Each region has a different amount of slots in each tier
205 responses per page so divide slots/page to get amount of requests 
if its not can use modulus operation to add one more page if the number has decimals



In [ ]:
def stats(queue):
    """ Takes rank queue and saves information of players returns list of seperate stats"""
    streak,summoner_name,points,veteran,wins,losses,games,win_percent=[],[],[],[],[],[],[],[]
    for summoner in queue:
        summoner_name.append(summoner['summonerName'])
        streak.append(summoner['hotStreak'])
        points.append(summoner['leaguePoints'])
        veteran.append(summoner['veteran'])
        win_percent.append(summoner['wins']/(summoner['losses']+summoner['wins'])*100)
        losses.append(summoner['losses'])
        wins.append(summoner['wins'])
        games.append(summoner['losses']+summoner['wins'])
    return streak,summoner_name,points,veteran,wins,losses,games,win_percent
#loop through challenger pages  compile dict of all challenger players make function
def get_players(url,pages):
  """
  accept api url as a string
  and number of pages as an int
  loops through pages collects all players and returns a list
  """
  dict_complete=[] #list created by combining all pages
  for x in range(pages):
    page=str(x+1)
    new_url=url+page
    print(new_url) #for logging
    r=requests.get(new_url,headers=headers)
    time.sleep(1.2)#avoid rate limit
    if r.status_code == 200:
      dict_work = r.json() #data on individual page
      dict_complete+=dict_work
  return dict_complete
import requests
import matplotlib.pyplot as plt
import time
from datetime import datetime
import pandas as pd

In [ ]:
#Each region has a prefix
prefix ='insert region prefix here ' 
Token = 'insert riot api token here'

challenger_url = 'https://'+prefix+'.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page='
grandmaster_url = 'https://'+prefix+'.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page='
master_url = 'https://'+prefix+'.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/MASTER/I?page='
headers={
    "Origin": "https://developer.riotgames.com",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "X-Riot-Token": Token,
    "Accept-Language": "en-US,en;q=0.9",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"
}


In [ ]:
# for NA,EUW and KR challenger has 2 pages grandmaster 4 and maser 20
challenger_dict=get_players(challenger_url,2)
grandmaster_dict=get_players(grandmaster_url,4)
master_dict=get_players(master_url,20)

In [ ]:
df_challenge =pd.DataFrame(challenger_dict)
df_grandmaster=pd.DataFrame(grandmaster_dict)
df_master=pd.DataFrame(master_dict)
# since concatanated dont use iloc
tiers=pd.concat([df_challenge,df_grandmaster,df_master])

In [ ]:
#loops through encrypted summoner ID and gets account ID
summoner_account=[]
get_account_start=time.time()
for  summonerId in tiers.summonerId:
    
    url = 'https://'+prefix+'.api.riotgames.com/lol/summoner/v4/summoners/'+summonerId
    r=requests.get(url,headers=headers)
    if r.status_code == 200:
        summoner =r.json()
        summoner_account.append(summoner['accountId'])
        
        time.sleep(1.2)# wait so as not to exceed rate limit 100 per 2 minutes loop takes on avg .6 seconds 120/100 = 1.2 .6+.6 = 1.2 seconds 
    elif r.status_code ==503: # 503 was commmon so just send request again
        r=requests.get(url,headers=headers)
        summoner =r.json()
        summoner_account.append(summoner['accountId'])
        time.sleep(1.2)
    else:
        print(f"Error summoner id:{summonerId}\nError status code:{r.status_code}")
        time.sleep(1.2) 
get_account_end = time.time()
get_account_time=get_account_end-get_account_start
print(get_account_time)
tiers['accountId']=summoner_account

In [ ]:
# Gets recent 100 Matches using encrypted match ID checks which role is their main role
roles=[]
get_roles_start=time.time()
for accountId in tiers.accountId:
    url = 'https://'+prefix+'.api.riotgames.com/lol/match/v4/matchlists/by-account/'+accountId +'?queue=420'
    r= requests.get(url,headers=headers)
    if r.status_code == 200:
        matches=r.json()
        df=pd.DataFrame(matches['matches'])
        top=jungle=mid=bot=support=autofill=0
        lane={}
        lane['Top']=len(df[df.lane=='TOP'])
        lane['Jungle']=len(df[df.lane=='JUNGLE'])
        lane['Mid']=len(df[df.lane=='MID'])
  
        lane['Bottom']=len(df[df.lane=='BOTTOM'])
        lane['Support']=len(df[df.lane=='NONE'])
        role = max(lane, key=lane.get)
        roles.append(role)
        time.sleep(1.2)
    elif r.status_code ==503: # 503 was commmon so just send request again
        r=requests.get(url,headers=headers)
        matches =r.json()
        df=pd.DataFrame(matches['matches'])
        top=jungle=mid=bot=support=autofill=0
        lane={}
        lane['Top']=len(df[df.lane=='TOP'])
        lane['Jungle']=len(df[df.lane=='JUNGLE'])
        lane['Mid']=len(df[df.lane=='MID'])
        lane['Bottom']=len(df[df.lane=='BOTTOM'])
        lane['Support']=len(df[df.lane=='NONE'])
        role = max(lane, key=lane.get)
        roles.append(role)
        time.sleep(1.2)
        
    else:
       print(f"For account:{accountId}\nError status code:{r.status_code}") 
       time.sleep(1.2)
get_roles_end=time.time()
get_roles_time=get_roles_end-get_roles_start
print(get_roles_time)
 tiers['role']=roles

In [ ]:
#create new data frame by dropping fluff information
tiers_working= tiers.drop(columns=['summonerId','accountId','queueType','leagueId','rank','leaguePoints','summonerName'])

In [ ]:
#calculate win rate
tiers_working_win_percent=pd.Series(round((tiers_working['wins']/(tiers_working['wins']+tiers_working['losses']))*100,2),name='winPercent')
tiers_working['winRate']=tiers_working_win_percent.value

In [ ]:
#calculate total games played
total_games=pd.Series(tiers_working['wins']+tiers_working['losses'])
tiers_working['totalGames']=total_games.values

In [ ]:
tiers_working_win_percent=pd.Series(round((tiers_working['wins']/(tiers_working['wins']+tiers_working['losses']))*100,2),name='winPercent')

In [ ]:
tiers_working.head()

In [ ]:
tiers_working['totalGames']=total_games.values

In [ ]:
#Save progress
time_now=datetime.now().strftime("%Y-%m-%d-%H-%M-%S") #Save date and time
tiers_working.to_csv(r'tiers_working_'+prefix+time_now+'.csv',index=False) #save file as csv with date and time as unique identifier
